Import Libraries

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import OneCycleLR
from torchvision import datasets
from torchsummary import summary

In [2]:
import sys
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook

In [3]:
!pip install albumentations==0.4.6

In [4]:
import albumentations as A

In [5]:
#import data
train = datasets.CIFAR10('./data', train=True, download=True)

Files already downloaded and verified


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%cd drive/MyDrive/ERA2/Session9

/content/drive/MyDrive/ERA2/Session9


In [8]:
import model9
from model9 import CifarNet

Finding the mean and std for normaliation of images

In [9]:
n = 0
s = np.zeros(3)
sq = np.zeros(3)
mean = np.zeros(3)
variance = np.zeros(3)
stand_dev = np.zeros(3)
count = 0

for data, l in tqdm_notebook(train):
  # print(np.array(data).shape)
  x = np.array(data)/255
  # print(x.shape)
  s += x.sum(axis=(0,1)) # this needs to be explained here (why 0,1). how does this work and why normalize and why not just scale to between 0 to 1 by dividing it by 255
  sq += np.sum(np.square(x), axis=(0,1))  # the squared sum is calculated here for what purpose?
  n += x.shape[0]*x.shape[1]
  # print(s)
  # print(n)

  mean+=s/n
  variance += np.square(s - mean)
  stand_dev += np.sqrt(variance/n)
  count+=1

# here mean is calculated across all the channels  and  not just one channel
mu = s/n
# std = np.sqrt((s/n - mu)/1024)
std = np.sqrt((sq/n - np.square(mu))) # how is this  equal to standard deviation?
print(mu, sq/n, std, n)
print(mean/count, stand_dev/count)

<ipython-input-9-aa641629515e>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data, l in tqdm_notebook(train):


  0%|          | 0/50000 [00:00<?, ?it/s]

[0.49139968 0.48215841 0.44653091] [0.30249857 0.29176174 0.26781806] [0.24703223 0.24348513 0.26158784] 51200000
[0.49196659 0.48229005 0.4461573 ] [227115.06139745 222630.84598793 205985.90869157]


In [10]:
import torchvision
class Cifar10SearchDataset(torchvision.datasets.CIFAR10):
    def __init__(self, root="~/data/cifar10", train=True, download=True, transform=None):
        super().__init__(root=root, train=train, download=download, transform=transform)

    def __getitem__(self, index):
        image, label = self.data[index], self.targets[index]

        if self.transform is not None:
            transformed = self.transform(image=image)
            image = transformed["image"]

        return image, label

In [11]:
from torchvision import transforms
from albumentations.pytorch.transforms import ToTensor

channel_means = (0.49196659, 0.48229005, 0.4461573)
channel_stdevs = (0.24703223, 0.24348513, 0.26158784)
fillmeans = tuple((np.array(channel_means)*255).astype(np.uint8))
# this is  to fill the empty spaces with  means of the channels multiplies by 255 to revert back to an image in that range
print(fillmeans, 'This is the value filled  at the time of rotation')
train_transforms = A.Compose([A.ShiftScaleRotate((-10.5,10.5)),
                              A.HorizontalFlip(),
                              A.CoarseDropout(max_holes = 1, max_height=16, max_width=16, min_holes = 1, min_height=4, min_width=4, fill_value=channel_means),
                              A.Normalize(mean=channel_means, std=channel_stdevs),
                              ToTensor(),
                              ])

# Test Phase transformations
test_transforms = A.Compose([A.Normalize(mean=channel_means, std=channel_stdevs), ToTensor(),])

(125, 122, 113) This is the value filled  at the time of rotation


In [12]:
train = Cifar10SearchDataset( train=True, download=True, transform=train_transforms)
test = Cifar10SearchDataset( train=False, download=True, transform=test_transforms)

100%|██████████| 170498071/170498071 [00:06<00:00, 28239107.23it/s]


Extracting /root/data/cifar10/cifar-10-python.tar.gz to /root/data/cifar10
Files already downloaded and verified


In [13]:
seed = 1
batch_size = 64

cuda = torch.cuda.is_available()
if cuda:
    torch.cuda.manual_seed(seed)
    dataloader_args = dict(shuffle=True, batch_size=batch_size, num_workers=4, pin_memory=True)

train_loader = torch.utils.data.DataLoader(train, **dataloader_args)
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [15]:
# from __future__ import print_function
# channel_means = (0.49196659, 0.48229005, 0.4461573)
# channel_stdevs = (0.24703223, 0.24348513, 0.26158784)
# def unnormalize(img):
#   img = img.numpy().astype(dtype=np.float32)

#   for i in range(img.shape[0]):
#     img[i] = (img[i]*channel_stdevs[i])+channel_means[i]

#   return np.transpose(img, (1,2,0))
#   #return np.transpose(img, (1, 2, 0))

# %matplotlib inline
# import matplotlib.pyplot as plt
# dataiter = iter(train_loader)
# images, labels = dataiter.next()

# print(images.shape)
# print(labels.shape)

# num_classes = 10
# # display 10 images from each category.
# class_names = ['airplane','automobile','bird','cat','deer',
#                'dog','frog','horse','ship','truck']
# r, c = 10, 11
# n = 5
# fig = plt.figure(figsize=(7,7))
# fig.subplots_adjust(hspace=0.01, wspace=0.01)
# for i in range(num_classes):
#     idx = np.random.choice(np.where(labels[:]==i)[0], n)
#     ax = plt.subplot(r, c, i*c+1)
#     ax.text(-1.5, 0.5, class_names[i], fontsize=14)
#     plt.axis('off')
#     for j in range(1, n+1):
#       plt.subplot(r, c, i*c+j+1)
#       plt.imshow(unnormalize(images[idx[j-1]]), interpolation='none')
#       plt.axis('off')
# plt.show()

In [16]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    data, target = data.to(device), target.to(device)
    optimizer.zero_grad()
    y_pred = model(data)
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)
    loss.backward()
    optimizer.step()
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)
    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))



In [19]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [20]:
model = CifarNet().to(device)
summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
              ReLU-2           [-1, 16, 32, 32]               0
       BatchNorm2d-3           [-1, 16, 32, 32]              32
            Conv2d-4           [-1, 16, 32, 32]           2,320
              ReLU-5           [-1, 16, 32, 32]               0
       BatchNorm2d-6           [-1, 16, 32, 32]              32
            Conv2d-7           [-1, 16, 16, 16]           2,320
            Conv2d-8           [-1, 16, 16, 16]             272
            Conv2d-9           [-1, 16, 16, 16]             160
           Conv2d-10           [-1, 32, 16, 16]             544
             ReLU-11           [-1, 32, 16, 16]               0
      BatchNorm2d-12           [-1, 32, 16, 16]              64
           Conv2d-13           [-1, 32, 16, 16]           9,248
             ReLU-14           [-1, 32,

In [21]:
EPOCHS = 250


optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.5, steps_per_epoch=len(train_loader), epochs=EPOCHS)


for epoch in range(EPOCHS):
  print("EPOCH:", epoch)
  train(model, device, train_loader, optimizer, epoch)
  scheduler.step()
  test(model, device, test_loader)

EPOCH: 0


Loss=1.9799057245254517 Batch_id=781 Accuracy=29.65: 100%|██████████| 782/782 [00:17<00:00, 44.66it/s]



Test set: Average loss: 1.5681, Accuracy: 4380/10000 (43.80%)

EPOCH: 1


Loss=1.5603795051574707 Batch_id=781 Accuracy=38.41: 100%|██████████| 782/782 [00:16<00:00, 46.05it/s]



Test set: Average loss: 1.3241, Accuracy: 5209/10000 (52.09%)

EPOCH: 2


...


Test set: Average loss: 0.4519, Accuracy: 8536/10000 (85.36%)

EPOCH: 248


Loss=0.452963262796402 Batch_id=781 Accuracy=76.38: 100%|██████████| 782/782 [00:21<00:00, 35.65it/s]



Test set: Average loss: 0.4320, Accuracy: 8609/10000 (86.09%)

EPOCH: 249


Loss=0.8964311480522156 Batch_id=781 Accuracy=76.40: 100%|██████████| 782/782 [00:21<00:00, 36.08it/s]



Test set: Average loss: 0.4258, Accuracy: 8628/10000 (86.28%)

